# Airfoil's Aerodynamic Coefficient Prediction using ANNs

In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [2]:
# importing the dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow import keras
from keras import Input
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
from keras.callbacks import ReduceLROnPlateau
import matplotlib.pyplot as plt

In [3]:
# reading the csv file
df = pd.read_csv("/home/nevilcp/ML_Aero/results/NACA4D_10/NACA4D_10.csv")

In [4]:
# printing the first 10 rows of the dataset
df.head(10)

,t,m,p,yU1,yU2,yU3,yU4,yU5,yU6,yU7,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
0,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-10.0,0.2,100000,-1.158,0.02657,0.005
1,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-9.0,0.2,100000,-1.043,0.02194,0.004
2,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-8.0,0.2,100000,-0.928,0.01876,0.004
3,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-7.0,0.2,100000,-0.813,0.01463,0.004
4,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-6.0,0.2,100000,-0.697,0.02806,0.003
5,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-5.0,0.2,100000,-0.581,0.01660,0.003
6,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-4.0,0.2,100000,-0.465,0.01146,0.002
7,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-3.0,0.2,100000,-0.349,0.00893,0.002
8,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-2.0,0.2,100000,-0.233,0.01393,0.001
9,5,0,5,0.009825,0.017881,0.023099,0.02499,0.023669,0.019965,0.014949,...,-0.014949,-0.009639,-0.004948,-0.001694,-1.0,0.2,100000,-0.116,0.01358,0.001


In [5]:
# checking for missing values in the dataset
df.isnull().sum()

t        0
m        0
p        0
yU1      0
yU2      0
yU3      0
yU4      0
yU5      0
yU6      0
yU7      0
yU8      0
yU9      0
yU10     0
yL1      0
yL2      0
yL3      0
yL4      0
yL5      0
yL6      0
yL7      0
yL8      0
yL9      0
yL10     0
alpha    0
M        0
Re       0
CL       0
CD       0
Cm       0
dtype: int64

In [6]:
# description of the dataset
df.describe()

,t,m,p,yU1,yU2,yU3,yU4,yU5,yU6,yU7,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
count,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,...,176400.000000,176400.000000,176400.000000,176400.000000,176400.000000,176400.00000,176400.000000,176400.000000,176400.000000,176400.000000
mean,20.000000,4.500000,40.000000,0.056883,0.096159,0.125194,0.138691,0.136083,0.120107,0.094865,...,-0.025367,-0.013472,-0.006526,-0.003090,0.000000,0.20000,300000.000000,0.692283,0.026646,-0.134151
std,10.000028,2.872289,22.912943,0.030652,0.043042,0.052297,0.056240,0.054535,0.048225,0.039002,...,0.037872,0.026573,0.014169,0.004346,6.055318,0.08165,141421.757093,0.926925,0.010960,0.113007
min,5.000000,0.000000,5.000000,0.009825,0.017881,0.023099,0.024990,0.023669,0.019965,0.014949,...,-0.104640,-0.067474,-0.034638,-0.011856,-10.000000,0.10000,100000.000000,-1.489000,0.001800,-0.514000
25%,10.000000,2.000000,22.500000,0.031423,0.060652,0.084089,0.094866,0.094677,0.083039,0.065317,...,-0.053389,-0.032844,-0.016811,-0.006339,-5.000000,0.10000,200000.000000,-0.011000,0.018990,-0.199000
50%,20.000000,4.500000,40.000000,0.054270,0.096111,0.124907,0.138240,0.136379,0.119788,0.094447,...,-0.025009,-0.014037,-0.006982,-0.003241,-0.000000,0.20000,300000.000000,0.691000,0.025390,-0.106000
75%,30.000000,7.000000,57.500000,0.078524,0.129458,0.166614,0.182079,0.176607,0.156303,0.123175,...,0.001474,0.004181,0.002635,-0.000235,5.000000,0.30000,400000.000000,1.379000,0.032540,-0.046000
max,35.000000,9.000000,75.000000,0.149810,0.208025,0.246146,0.264431,0.256034,0.230429,0.194578,...,0.074748,0.071348,0.042540,0.011816,10.000000,0.30000,500000.000000,3.599000,0.235470,0.025000


In [7]:
# shuffling all the rows
df = df.sample(frac=1)
df.head(10)

,t,m,p,yU1,yU2,yU3,yU4,yU5,yU6,yU7,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
166561,25,9,35,0.083044,0.140606,0.187630,0.212391,0.207767,0.181848,0.141052,...,-0.010638,-0.005517,-0.003314,-0.002642,-0.0,0.1,200000,1.291,0.03152,-0.226
86970,20,4,75,0.045692,0.082135,0.110061,0.125262,0.127115,0.117294,0.099533,...,-0.019914,-0.001880,0.001966,-0.000470,-1.0,0.1,200000,0.748,0.02337,-0.217
119702,15,6,65,0.037223,0.070159,0.098474,0.117138,0.123946,0.118853,0.103448,...,0.013758,0.015862,0.009310,0.001586,-8.0,0.1,100000,0.056,0.02257,-0.249
37586,5,2,15,0.015638,0.033713,0.043080,0.044427,0.041521,0.035069,0.026359,...,-0.003550,-0.002336,-0.001381,-0.000750,7.0,0.2,500000,1.023,0.01410,-0.037
90516,5,5,15,0.024975,0.057759,0.073045,0.073590,0.068316,0.057757,0.043509,...,0.013523,0.008594,0.003950,0.000656,-4.0,0.3,100000,0.056,0.01610,-0.085
90684,10,5,15,0.038529,0.077002,0.096061,0.098605,0.092105,0.077937,0.058694,...,-0.001332,-0.000960,-0.000941,-0.001013,-4.0,0.2,400000,0.067,0.01429,-0.080
64760,15,3,55,0.034079,0.063137,0.085894,0.098462,0.099461,0.089824,0.071413,...,-0.018324,-0.010089,-0.005021,-0.002369,7.0,0.3,400000,1.390,0.02455,-0.123
75426,10,4,25,0.029112,0.058995,0.082763,0.089848,0.085119,0.072756,0.055215,...,-0.004708,-0.002913,-0.001824,-0.001235,5.0,0.2,200000,1.074,0.02024,-0.087
105026,25,5,75,0.059190,0.104280,0.138294,0.156693,0.158760,0.146459,0.124361,...,-0.024917,-0.001916,0.002709,-0.000535,-5.0,0.1,200000,0.461,0.02651,-0.263
106542,15,6,5,0.083217,0.111196,0.127535,0.131249,0.122262,0.103014,0.077247,...,-0.013393,-0.008894,-0.005092,-0.002484,-1.0,0.1,400000,0.535,0.00786,-0.067


In [8]:
# dropping columns t, m & p
df = df.drop(columns=['t', 'm', 'p'], axis=1)
df.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL7,yL8,yL9,yL10,alpha,M,Re,CL,CD,Cm
166561,0.083044,0.140606,0.187630,0.212391,0.207767,0.181848,0.141052,0.093227,0.047670,0.014655,...,-0.010638,-0.005517,-0.003314,-0.002642,-0.0,0.1,200000,1.291,0.03152,-0.226
86970,0.045692,0.082135,0.110061,0.125262,0.127115,0.117294,0.099533,0.075388,0.042243,0.013407,...,-0.019914,-0.001880,0.001966,-0.000470,-1.0,0.1,200000,0.748,0.02337,-0.217
119702,0.037223,0.070159,0.098474,0.117138,0.123946,0.118853,0.103448,0.074478,0.039891,0.012115,...,0.013758,0.015862,0.009310,0.001586,-8.0,0.1,100000,0.056,0.02257,-0.249
37586,0.015638,0.033713,0.043080,0.044427,0.041521,0.035069,0.026359,0.016955,0.008522,0.002639,...,-0.003550,-0.002336,-0.001381,-0.000750,7.0,0.2,500000,1.023,0.01410,-0.037
90516,0.024975,0.057759,0.073045,0.073590,0.068316,0.057757,0.043509,0.027957,0.013899,0.004060,...,0.013523,0.008594,0.003950,0.000656,-4.0,0.3,100000,0.056,0.01610,-0.085
90684,0.038529,0.077002,0.096061,0.098605,0.092105,0.077937,0.058694,0.037802,0.018966,0.005787,...,-0.001332,-0.000960,-0.000941,-0.001013,-4.0,0.2,400000,0.067,0.01429,-0.080
64760,0.034079,0.063137,0.085894,0.098462,0.099461,0.089824,0.071413,0.047898,0.024786,0.007810,...,-0.018324,-0.010089,-0.005021,-0.002369,7.0,0.3,400000,1.390,0.02455,-0.123
75426,0.029112,0.058995,0.082763,0.089848,0.085119,0.072756,0.055215,0.035773,0.018043,0.005554,...,-0.004708,-0.002913,-0.001824,-0.001235,5.0,0.2,200000,1.074,0.02024,-0.087
105026,0.059190,0.104280,0.138294,0.156693,0.158760,0.146459,0.124361,0.094786,0.053619,0.017090,...,-0.024917,-0.001916,0.002709,-0.000535,-5.0,0.1,200000,0.461,0.02651,-0.263
106542,0.083217,0.111196,0.127535,0.131249,0.122262,0.103014,0.077247,0.049666,0.024973,0.007757,...,-0.013393,-0.008894,-0.005092,-0.002484,-1.0,0.1,400000,0.535,0.00786,-0.067


In [9]:
# defining the feature and target columns
X = df.drop(columns=['CL', 'CD', 'Cm'])
y = df[['CL', 'CD', 'Cm']]

In [10]:
# displaying the features table
X.head(10)

,yU1,yU2,yU3,yU4,yU5,yU6,yU7,yU8,yU9,yU10,...,yL4,yL5,yL6,yL7,yL8,yL9,yL10,alpha,M,Re
166561,0.083044,0.140606,0.187630,0.212391,0.207767,0.181848,0.141052,0.093227,0.047670,0.014655,...,-0.038184,-0.029600,-0.019342,-0.010638,-0.005517,-0.003314,-0.002642,-0.0,0.1,200000
86970,0.045692,0.082135,0.110061,0.125262,0.127115,0.117294,0.099533,0.075388,0.042243,0.013407,...,-0.075043,-0.062415,-0.042383,-0.019914,-0.001880,0.001966,-0.000470,-1.0,0.1,200000
119702,0.037223,0.070159,0.098474,0.117138,0.123946,0.118853,0.103448,0.074478,0.039891,0.012115,...,-0.033512,-0.018370,-0.000929,0.013758,0.015862,0.009310,0.001586,-8.0,0.1,100000
37586,0.015638,0.033713,0.043080,0.044427,0.041521,0.035069,0.026359,0.016955,0.008522,0.002639,...,-0.005555,-0.005823,-0.004870,-0.003550,-0.002336,-0.001381,-0.000750,7.0,0.2,500000
90516,0.024975,0.057759,0.073045,0.073590,0.068316,0.057757,0.043509,0.027957,0.013899,0.004060,...,0.023599,0.020939,0.017752,0.013523,0.008594,0.003950,0.000656,-4.0,0.3,100000
90684,0.038529,0.077002,0.096061,0.098605,0.092105,0.077937,0.058694,0.037802,0.018966,0.005787,...,-0.001379,-0.002698,-0.002139,-0.001332,-0.000960,-0.000941,-0.001013,-4.0,0.2,400000
64760,0.034079,0.063137,0.085894,0.098462,0.099461,0.089824,0.071413,0.047898,0.024786,0.007810,...,-0.051660,-0.042592,-0.029943,-0.018324,-0.010089,-0.005021,-0.002369,7.0,0.3,400000
75426,0.029112,0.058995,0.082763,0.089848,0.085119,0.072756,0.055215,0.035773,0.018043,0.005554,...,-0.010116,-0.009599,-0.007204,-0.004708,-0.002913,-0.001824,-0.001235,5.0,0.2,200000
105026,0.059190,0.104280,0.138294,0.156693,0.158760,0.146459,0.124361,0.094786,0.053619,0.017090,...,-0.093879,-0.078346,-0.053196,-0.024917,-0.001916,0.002709,-0.000535,-5.0,0.1,200000
106542,0.083217,0.111196,0.127535,0.131249,0.122262,0.103014,0.077247,0.049666,0.024973,0.007757,...,-0.018714,-0.020480,-0.017810,-0.013393,-0.008894,-0.005092,-0.002484,-1.0,0.1,400000


In [11]:
# displaying the targets table
y.head(10)

,CL,CD,Cm
166561,1.291,0.03152,-0.226
86970,0.748,0.02337,-0.217
119702,0.056,0.02257,-0.249
37586,1.023,0.01410,-0.037
90516,0.056,0.01610,-0.085
90684,0.067,0.01429,-0.080
64760,1.390,0.02455,-0.123
75426,1.074,0.02024,-0.087
105026,0.461,0.02651,-0.263
106542,0.535,0.00786,-0.067


## Splitting the Dataset

In [12]:
# splitting dataset into training, validation & testing dataset
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.7, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [13]:
# normalizing the training, validation & testing datasets
X_scaler = StandardScaler()
X_train = X_scaler.fit_transform(X_train)
X_val = X_scaler.transform(X_val)
X_test = X_scaler.transform(X_test)

In [14]:
# displaying X_train, X_val & X_test after standardization
display(pd.DataFrame(X_train).head())
display(pd.DataFrame(X_val).head())
display(pd.DataFrame(X_test).head())

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,1.239136,1.360341,1.415372,1.487935,1.576398,1.657706,1.677922,1.592884,1.516341,1.536504,...,-1.195661,-1.036602,-0.829987,-0.640920,-0.529724,-0.487333,-0.612694,-0.990656,1.223499,1.416735
1,-0.138517,-0.008894,0.136744,0.330998,0.583513,0.914023,1.394478,2.050286,2.417806,2.326441,...,0.303572,0.575812,0.936140,1.458160,2.139663,2.404963,2.155113,0.164526,-0.001220,-0.708695
2,-1.127245,-1.232131,-1.242594,-1.206647,-1.143469,-1.052088,-0.917760,-0.809291,-0.747570,-0.784992,...,0.576319,0.605068,0.626919,0.652062,0.623120,0.617902,0.709650,-1.320709,1.223499,1.416735
3,-1.538277,-1.821759,-1.955169,-2.024418,-2.063564,-2.078280,-2.050176,-1.954061,-1.854438,-1.827126,...,0.650059,0.554118,0.425924,0.275147,0.144625,0.111909,0.322179,1.649761,-1.225938,-1.417171
4,-0.064209,0.023616,0.037404,0.032993,0.019276,-0.007506,-0.055706,-0.114453,-0.135206,-0.058782,...,-0.852211,-0.840731,-0.832504,-0.828366,-0.818791,-0.800652,-0.784170,0.494578,-0.001220,0.708259


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,1.653620,1.729051,1.797634,1.902872,2.005093,2.042285,1.986845,1.831367,1.704754,1.679601,...,-0.792477,-0.609534,-0.462993,-0.366934,-0.328781,-0.333834,-0.496777,0.989657,-0.001220,-0.708695
1,-0.575125,-0.574030,-0.628916,-0.690404,-0.760888,-0.836029,-0.900301,-0.922954,-0.898505,-0.792752,...,-0.683884,-0.751830,-0.833784,-0.909007,-0.944419,-0.937052,-0.848270,0.164526,-1.225938,-0.708695
2,0.824064,1.020731,1.058314,1.063132,1.057489,1.030715,0.960157,0.841440,0.772190,0.874933,...,-1.603188,-1.544449,-1.460495,-1.363449,-1.270749,-1.215174,-1.273723,1.154683,-0.001220,-0.708695
3,0.563808,0.668461,0.502351,0.349067,0.237267,0.134146,0.017470,-0.092913,-0.158576,-0.170975,...,0.355239,0.234676,0.125434,0.006415,-0.101837,-0.158389,-0.162233,-0.000500,1.223499,-1.417171
4,1.779329,1.851656,1.933353,2.060590,2.188620,2.248805,2.210287,2.056036,1.924103,1.871777,...,-0.644667,-0.430675,-0.260666,-0.155197,-0.123736,-0.141190,-0.332375,0.989657,1.223499,-0.708695


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,2.794359,2.057226,2.024481,1.976240,1.880930,1.742506,1.542104,1.296008,1.134206,1.156649,...,-0.658481,-0.827650,-0.916872,-0.971554,-0.995362,-1.014255,-1.120451,-0.000500,-0.00122,0.708259
1,-0.387621,-0.263228,-0.187017,-0.248159,-0.326220,-0.392772,-0.462702,-0.519341,-0.548147,-0.574208,...,0.647498,0.551785,0.449015,0.321531,0.194957,0.133065,0.172293,-0.495578,-0.00122,-0.000218
2,1.211850,1.160642,0.989639,0.890901,0.784986,0.661932,0.508368,0.353001,0.268917,0.346377,...,-0.880116,-0.965998,-1.030629,-1.078005,-1.091649,-1.092552,-1.127677,1.154683,-0.00122,-0.000218
3,-0.671312,-0.446603,-0.285683,-0.338139,-0.415513,-0.470695,-0.527931,-0.577303,-0.613755,-0.707729,...,1.446801,1.333278,1.209572,1.042527,0.857043,0.755886,0.791093,0.329552,-0.00122,0.708259
4,0.345175,0.615074,0.449098,0.251093,0.127627,0.027617,-0.080578,-0.184122,-0.255970,-0.332649,...,1.145718,0.998471,0.860889,0.697306,0.528232,0.431761,0.424637,-0.825630,-0.00122,1.416735


In [15]:
# displaying y_train, y_val & y_test after standardization
display(pd.DataFrame(y_train).head())
display(pd.DataFrame(y_val).head())
display(pd.DataFrame(y_test).head())

,CL,CD,Cm
119053,0.265,0.03112,-0.220
175277,2.027,0.03793,-0.484
66656,-0.467,0.01706,-0.127
125,1.140,0.02494,-0.005
47641,0.757,0.02451,-0.082


,CL,CD,Cm
151972,2.243,0.05420,-0.263
1187,0.129,0.02018,-0.002
65978,1.585,0.04308,-0.132
109210,0.768,0.02675,-0.094
169717,2.473,0.06213,-0.300


,CL,CD,Cm
143188,1.083,0.01255,-0.088
93289,0.240,0.01683,-0.101
74504,1.541,0.03771,-0.065
128385,1.060,0.01807,-0.146
144254,0.297,0.01587,-0.128


In [16]:
# defining the learning rate reduction callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=5, min_lr=0.00001)

## Network Performance Due to Multiple Hidden Layers

### Case 1: Model [64, 3]

In [17]:
# RMSE and R² metric scores on test data for Model M_64
M_64_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64...")

    M_64 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_64.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_64_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_df = pd.DataFrame(M_64_metrics)

print(results_M_64_df.mean())



Run 1/20 for Model M_64...


I0000 00:00:1764161435.284624  669121 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6155 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9
I0000 00:00:1764161437.652259  669252 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.



Run 2/20 for Model M_64...

Run 3/20 for Model M_64...

Run 4/20 for Model M_64...

Run 5/20 for Model M_64...

Run 6/20 for Model M_64...

Run 7/20 for Model M_64...

Run 8/20 for Model M_64...

Run 9/20 for Model M_64...

Run 10/20 for Model M_64...

Run 11/20 for Model M_64...

Run 12/20 for Model M_64...

Run 13/20 for Model M_64...

Run 14/20 for Model M_64...

Run 15/20 for Model M_64...

Run 16/20 for Model M_64...

Run 17/20 for Model M_64...

Run 18/20 for Model M_64...

Run 19/20 for Model M_64...

Run 20/20 for Model M_64...
CL_rmse    0.005312
CD_rmse    0.004230
Cm_rmse    0.002296
CL_r2      0.999967
CD_r2      0.846132
Cm_r2      0.999583
dtype: float32


### Case 2: Model [64, 32, 3]

In [18]:
# RMSE and R² metric scores on test data for Model M_64_32
M_64_32_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32...")
    
    M_64_32 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )
    
    M_64_32.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    
    M_64_32_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_df = pd.DataFrame(M_64_32_metrics)

print(results_M_64_32_df.mean())



Run 1/20 for Model M_64_32...

Run 2/20 for Model M_64_32...

Run 3/20 for Model M_64_32...

Run 4/20 for Model M_64_32...

Run 5/20 for Model M_64_32...

Run 6/20 for Model M_64_32...

Run 7/20 for Model M_64_32...

Run 8/20 for Model M_64_32...

Run 9/20 for Model M_64_32...

Run 10/20 for Model M_64_32...

Run 11/20 for Model M_64_32...

Run 12/20 for Model M_64_32...

Run 13/20 for Model M_64_32...

Run 14/20 for Model M_64_32...

Run 15/20 for Model M_64_32...

Run 16/20 for Model M_64_32...

Run 17/20 for Model M_64_32...

Run 18/20 for Model M_64_32...

Run 19/20 for Model M_64_32...

Run 20/20 for Model M_64_32...
CL_rmse    0.004231
CD_rmse    0.003978
Cm_rmse    0.001881
CL_r2      0.999979
CD_r2      0.863597
Cm_r2      0.999721
dtype: float32


### Case 3: Model [64, 32, 16, 3]

In [19]:
# RMSE and R² metric scores on test data for Model M_64_32_16
M_64_32_16_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32_16...")
    
    M_64_32_16 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32_16.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )
    
    M_64_32_16.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32_16.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    
    M_64_32_16_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_16_df = pd.DataFrame(M_64_32_16_metrics)

print(results_M_64_32_16_df.mean())



Run 1/20 for Model M_64_32_16...

Run 2/20 for Model M_64_32_16...

Run 3/20 for Model M_64_32_16...

Run 4/20 for Model M_64_32_16...

Run 5/20 for Model M_64_32_16...

Run 6/20 for Model M_64_32_16...

Run 7/20 for Model M_64_32_16...

Run 8/20 for Model M_64_32_16...

Run 9/20 for Model M_64_32_16...

Run 10/20 for Model M_64_32_16...

Run 11/20 for Model M_64_32_16...

Run 12/20 for Model M_64_32_16...

Run 13/20 for Model M_64_32_16...

Run 14/20 for Model M_64_32_16...

Run 15/20 for Model M_64_32_16...

Run 16/20 for Model M_64_32_16...

Run 17/20 for Model M_64_32_16...

Run 18/20 for Model M_64_32_16...

Run 19/20 for Model M_64_32_16...

Run 20/20 for Model M_64_32_16...
CL_rmse    0.004128
CD_rmse    0.003866
Cm_rmse    0.001939
CL_r2      0.999980
CD_r2      0.871552
Cm_r2      0.999701
dtype: float32


### Case 4: Model [64, 32, 16, 8, 3]

In [20]:
# RMSE and R² metric scores on test data for Model M_64_32_16_8
M_64_32_16_8_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32_16_8...")
    
    M_64_32_16_8 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(8, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32_16_8.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )
    
    M_64_32_16_8.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32_16_8.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))
    
    M_64_32_16_8_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_16_8_df = pd.DataFrame(M_64_32_16_8_metrics)

print(results_M_64_32_16_8_df.mean())



Run 1/20 for Model M_64_32_16_8...

Run 2/20 for Model M_64_32_16_8...

Run 3/20 for Model M_64_32_16_8...

Run 4/20 for Model M_64_32_16_8...

Run 5/20 for Model M_64_32_16_8...

Run 6/20 for Model M_64_32_16_8...

Run 7/20 for Model M_64_32_16_8...

Run 8/20 for Model M_64_32_16_8...

Run 9/20 for Model M_64_32_16_8...

Run 10/20 for Model M_64_32_16_8...

Run 11/20 for Model M_64_32_16_8...

Run 12/20 for Model M_64_32_16_8...

Run 13/20 for Model M_64_32_16_8...

Run 14/20 for Model M_64_32_16_8...

Run 15/20 for Model M_64_32_16_8...

Run 16/20 for Model M_64_32_16_8...

Run 17/20 for Model M_64_32_16_8...

Run 18/20 for Model M_64_32_16_8...

Run 19/20 for Model M_64_32_16_8...

Run 20/20 for Model M_64_32_16_8...
CL_rmse    0.003975
CD_rmse    0.003922
Cm_rmse    0.001817
CL_r2      0.999981
CD_r2      0.867454
Cm_r2      0.999738
dtype: float32


In [21]:
M_64_stats = results_M_64_df.mean()
M_64_32_stats = results_M_64_32_df.mean()
M_64_32_16_stats = results_M_64_32_16_df.mean()
M_64_32_16_8_stats = results_M_64_32_16_8_df.mean()

architectures = [
    '64, 3',
    '64, 32, 3',
    '64, 32, 16, 3',
    '64, 32, 16, 8, 3'
]

table_data = [
    (M_64_stats['CL_rmse'], M_64_stats['CD_rmse'], M_64_stats['Cm_rmse'],
     M_64_stats['CL_r2'], M_64_stats['CD_r2'], M_64_stats['Cm_r2']),
    
    (M_64_32_stats['CL_rmse'], M_64_32_stats['CD_rmse'], M_64_32_stats['Cm_rmse'],
     M_64_32_stats['CL_r2'], M_64_32_stats['CD_r2'], M_64_32_stats['Cm_r2']),
    
    (M_64_32_16_stats['CL_rmse'], M_64_32_16_stats['CD_rmse'], M_64_32_16_stats['Cm_rmse'],
     M_64_32_16_stats['CL_r2'], M_64_32_16_stats['CD_r2'], M_64_32_16_stats['Cm_r2']),
    
    (M_64_32_16_8_stats['CL_rmse'], M_64_32_16_8_stats['CD_rmse'], M_64_32_16_8_stats['Cm_rmse'],
     M_64_32_16_8_stats['CL_r2'], M_64_32_16_8_stats['CD_r2'], M_64_32_16_8_stats['Cm_r2'])
]

column_headers = pd.MultiIndex.from_tuples([
    ('RMSE', 'CL'), ('RMSE', 'CD'), ('RMSE', 'Cm'),
    ('R²', 'CL'), ('R²', 'CD'), ('R²', 'Cm')
])

row_index = pd.MultiIndex.from_tuples(
    zip(range(1, 5), architectures),
    names=['Case No.', 'Network Architecture']
)

table_1_df = pd.DataFrame(table_data, index=row_index, columns=column_headers)

display(table_1_df.style.set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'left')]},
     {'selector': 'td', 'props': [('text-align', 'left')]}]
).format(precision=6))


## Network Performance Due to Increasing Neurons

### Case 1: Model [64, 32, 16, 3]

In [22]:
# RMSE and R² metric scores on test data for Model M_64_32_16
M_64_32_16_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_64_32_16...")

    M_64_32_16 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_64_32_16.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_64_32_16.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_64_32_16.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_64_32_16_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_64_32_16_df = pd.DataFrame(M_64_32_16_metrics)

print(results_M_64_32_16_df.mean())



Run 1/20 for Model M_64_32_16...

Run 2/20 for Model M_64_32_16...

Run 3/20 for Model M_64_32_16...

Run 4/20 for Model M_64_32_16...

Run 5/20 for Model M_64_32_16...

Run 6/20 for Model M_64_32_16...

Run 7/20 for Model M_64_32_16...

Run 8/20 for Model M_64_32_16...

Run 9/20 for Model M_64_32_16...

Run 10/20 for Model M_64_32_16...

Run 11/20 for Model M_64_32_16...

Run 12/20 for Model M_64_32_16...

Run 13/20 for Model M_64_32_16...

Run 14/20 for Model M_64_32_16...

Run 15/20 for Model M_64_32_16...

Run 16/20 for Model M_64_32_16...

Run 17/20 for Model M_64_32_16...

Run 18/20 for Model M_64_32_16...

Run 19/20 for Model M_64_32_16...

Run 20/20 for Model M_64_32_16...
CL_rmse    0.004142
CD_rmse    0.003848
Cm_rmse    0.001800
CL_r2      0.999980
CD_r2      0.872731
Cm_r2      0.999745
dtype: float32


### Case 2: Model [128, 64, 32, 3]

In [23]:
# RMSE and R² metric scores on test data for Model M_128_64_32
M_128_64_32_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_128_64_32...")

    M_128_64_32 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_128_64_32.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_128_64_32.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_128_64_32.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_128_64_32_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_128_64_32_df = pd.DataFrame(M_128_64_32_metrics)

print(results_M_128_64_32_df.mean())



Run 1/20 for Model M_128_64_32...

Run 2/20 for Model M_128_64_32...

Run 3/20 for Model M_128_64_32...

Run 4/20 for Model M_128_64_32...

Run 5/20 for Model M_128_64_32...

Run 6/20 for Model M_128_64_32...

Run 7/20 for Model M_128_64_32...

Run 8/20 for Model M_128_64_32...

Run 9/20 for Model M_128_64_32...

Run 10/20 for Model M_128_64_32...

Run 11/20 for Model M_128_64_32...

Run 12/20 for Model M_128_64_32...

Run 13/20 for Model M_128_64_32...

Run 14/20 for Model M_128_64_32...

Run 15/20 for Model M_128_64_32...

Run 16/20 for Model M_128_64_32...

Run 17/20 for Model M_128_64_32...

Run 18/20 for Model M_128_64_32...

Run 19/20 for Model M_128_64_32...

Run 20/20 for Model M_128_64_32...
CL_rmse    0.003152
CD_rmse    0.003521
Cm_rmse    0.001312
CL_r2      0.999988
CD_r2      0.893397
Cm_r2      0.999864
dtype: float32


### Case 3: Model [256, 128, 64, 3]

In [24]:
# RMSE and R² metric scores on test data for Model M_256_128_64
M_256_128_64_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_256_128_64...")

    M_256_128_64 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_256_128_64.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_256_128_64.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_256_128_64.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_256_128_64_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_256_128_64_df = pd.DataFrame(M_256_128_64_metrics)

print(results_M_256_128_64_df.mean())



Run 1/20 for Model M_256_128_64...

Run 2/20 for Model M_256_128_64...

Run 3/20 for Model M_256_128_64...

Run 4/20 for Model M_256_128_64...

Run 5/20 for Model M_256_128_64...

Run 6/20 for Model M_256_128_64...

Run 7/20 for Model M_256_128_64...

Run 8/20 for Model M_256_128_64...

Run 9/20 for Model M_256_128_64...

Run 10/20 for Model M_256_128_64...

Run 11/20 for Model M_256_128_64...

Run 12/20 for Model M_256_128_64...

Run 13/20 for Model M_256_128_64...

Run 14/20 for Model M_256_128_64...

Run 15/20 for Model M_256_128_64...

Run 16/20 for Model M_256_128_64...

Run 17/20 for Model M_256_128_64...

Run 18/20 for Model M_256_128_64...

Run 19/20 for Model M_256_128_64...

Run 20/20 for Model M_256_128_64...
CL_rmse    0.002699
CD_rmse    0.003251
Cm_rmse    0.000991
CL_r2      0.999990
CD_r2      0.909062
Cm_r2      0.999923
dtype: float32


### Case 4: Model [512, 256, 128, 3]

In [25]:
# RMSE and R² metric scores on test data for Model M_512_256_128
M_512_256_128_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_512_256_128...")

    M_512_256_128 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_512_256_128.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_512_256_128.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_512_256_128.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_512_256_128_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_512_256_128_df = pd.DataFrame(M_512_256_128_metrics)

print(results_M_512_256_128_df.mean())



Run 1/20 for Model M_512_256_128...

Run 2/20 for Model M_512_256_128...

Run 3/20 for Model M_512_256_128...

Run 4/20 for Model M_512_256_128...

Run 5/20 for Model M_512_256_128...

Run 6/20 for Model M_512_256_128...

Run 7/20 for Model M_512_256_128...

Run 8/20 for Model M_512_256_128...

Run 9/20 for Model M_512_256_128...

Run 10/20 for Model M_512_256_128...

Run 11/20 for Model M_512_256_128...

Run 12/20 for Model M_512_256_128...

Run 13/20 for Model M_512_256_128...

Run 14/20 for Model M_512_256_128...

Run 15/20 for Model M_512_256_128...

Run 16/20 for Model M_512_256_128...

Run 17/20 for Model M_512_256_128...

Run 18/20 for Model M_512_256_128...

Run 19/20 for Model M_512_256_128...

Run 20/20 for Model M_512_256_128...
CL_rmse    0.002267
CD_rmse    0.003147
Cm_rmse    0.000868
CL_r2      0.999994
CD_r2      0.914501
Cm_r2      0.999940
dtype: float32


### Case 5: Model [1024, 512, 256, 3]

In [26]:
# RMSE and R² metric scores on test data for Model M_1024_512_256
M_1024_512_256_metrics = []

for i in range(20):
    print(f"\nRun {i+1}/20 for Model M_1024_512_256...")

    M_1024_512_256 = Sequential([
        Input(shape=(X_train.shape[1],)),
        Dense(1024, activation='relu'),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(y_train.shape[1], activation='linear')
    ])

    M_1024_512_256.compile(
        optimizer=Adam(learning_rate=5e-4, beta_1=0.9, beta_2=0.999),
        loss='mse',
        metrics=[RootMeanSquaredError()]
    )

    M_1024_512_256.fit(
        X_train, y_train,
        epochs=50, batch_size=128,
        validation_data=(X_val, y_val),
        callbacks=[reduce_lr],
        verbose=0
    )

    y_pred = M_1024_512_256.predict(X_test, verbose=0)
    r2 = r2_score(y_test, y_pred, multioutput='raw_values')
    rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))

    M_1024_512_256_metrics.append({
        'CL_rmse': rmse[0], 'CD_rmse': rmse[1], 'Cm_rmse': rmse[2],
        'CL_r2': r2[0], 'CD_r2': r2[1], 'Cm_r2': r2[2]
    })

results_M_1024_512_256_df = pd.DataFrame(M_1024_512_256_metrics)

print(results_M_1024_512_256_df.mean())



Run 1/20 for Model M_1024_512_256...

Run 2/20 for Model M_1024_512_256...

Run 3/20 for Model M_1024_512_256...

Run 4/20 for Model M_1024_512_256...

Run 5/20 for Model M_1024_512_256...

Run 6/20 for Model M_1024_512_256...

Run 7/20 for Model M_1024_512_256...

Run 8/20 for Model M_1024_512_256...

Run 9/20 for Model M_1024_512_256...

Run 10/20 for Model M_1024_512_256...

Run 11/20 for Model M_1024_512_256...

Run 12/20 for Model M_1024_512_256...

Run 13/20 for Model M_1024_512_256...

Run 14/20 for Model M_1024_512_256...

Run 15/20 for Model M_1024_512_256...

Run 16/20 for Model M_1024_512_256...

Run 17/20 for Model M_1024_512_256...

Run 18/20 for Model M_1024_512_256...

Run 19/20 for Model M_1024_512_256...

Run 20/20 for Model M_1024_512_256...
CL_rmse    0.002266
CD_rmse    0.003086
Cm_rmse    0.000864
CL_r2      0.999993
CD_r2      0.917466
Cm_r2      0.999939
dtype: float32


In [27]:
M_64_32_16_stats = results_M_64_32_16_df.mean()
M_128_64_32_stats = results_M_128_64_32_df.mean()
M_256_128_64_stats = results_M_256_128_64_df.mean()
M_512_256_128_stats = results_M_512_256_128_df.mean()
M_1024_512_256_stats = results_M_1024_512_256_df.mean()

architectures = [
    '64, 32, 16, 3',
    '128, 64, 32, 3',
    '256, 128, 64, 3',
    '512, 256, 128, 3',
    '1024, 512, 256, 3'
]

table_data = [
    (M_64_32_16_stats['CL_rmse'], M_64_32_16_stats['CD_rmse'], M_64_32_16_stats['Cm_rmse'],
     M_64_32_16_stats['CL_r2'], M_64_32_16_stats['CD_r2'], M_64_32_16_stats['Cm_r2']),
    
    (M_128_64_32_stats['CL_rmse'], M_128_64_32_stats['CD_rmse'], M_128_64_32_stats['Cm_rmse'],
     M_128_64_32_stats['CL_r2'], M_128_64_32_stats['CD_r2'], M_128_64_32_stats['Cm_r2']),
    
    (M_256_128_64_stats['CL_rmse'], M_256_128_64_stats['CD_rmse'], M_256_128_64_stats['Cm_rmse'],
     M_256_128_64_stats['CL_r2'], M_256_128_64_stats['CD_r2'], M_256_128_64_stats['Cm_r2']),
    
    (M_512_256_128_stats['CL_rmse'], M_512_256_128_stats['CD_rmse'], M_512_256_128_stats['Cm_rmse'],
     M_512_256_128_stats['CL_r2'], M_512_256_128_stats['CD_r2'], M_512_256_128_stats['Cm_r2']),
    
    (M_1024_512_256_stats['CL_rmse'], M_1024_512_256_stats['CD_rmse'], M_1024_512_256_stats['Cm_rmse'],
     M_1024_512_256_stats['CL_r2'], M_1024_512_256_stats['CD_r2'], M_1024_512_256_stats['Cm_r2'])
]

column_headers = pd.MultiIndex.from_tuples([
    ('RMSE', 'CL'), ('RMSE', 'CD'), ('RMSE', 'Cm'),
    ('R²', 'CL'), ('R²', 'CD'), ('R²', 'Cm')
])

row_index = pd.MultiIndex.from_tuples(
    zip(range(1, 6), architectures),
    names=['Case No.', 'Network Architecture']
)

table_1_df = pd.DataFrame(table_data, index=row_index, columns=column_headers)

display(table_1_df.style.set_table_styles(
    [{'selector': 'th', 'props': [('text-align', 'left')]},
     {'selector': 'td', 'props': [('text-align', 'left')]}]
).format(precision=6))
